In [1]:
# import libraries

import numpy as np
import pandas as pd
import requests
from datetime import timedelta, datetime
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
from sklearn.model_selection import train_test_split
from scipy import stats
import re
# import functions

from acquire import read_csv
from prepare import missing_values_table, handle_missing_values

In [2]:
df = read_csv()

In [3]:
df.head(2)

,Month,State,Genus Species,Serotype or Genotype,Etiology Status,Location of Preparation,Illnesses,Hospitalizations,Deaths,Food Vehicle,Contaminated Ingredient
Year,,,,,,,,,,,
2009,1,Minnesota,Norovirus,NaN,Suspected,Restaurant - Sit-down dining,2,0.0,0.0,NaN,NaN
2009,1,Minnesota,Norovirus,NaN,Confirmed,NaN,16,0.0,0.0,NaN,NaN


In [4]:
df.shape

(19119, 11)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19119 entries, 2009 to 2015
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Month                    19119 non-null  int64  
 1   State                    19119 non-null  object 
 2   Genus Species            12500 non-null  object 
 3   Serotype or Genotype     3907 non-null   object 
 4   Etiology Status          12500 non-null  object 
 5   Location of Preparation  17984 non-null  object 
 6   Illnesses                19119 non-null  int64  
 7   Hospitalizations         15494 non-null  float64
 8   Deaths                   15518 non-null  float64
 9   Food Vehicle             10156 non-null  object 
 10  Contaminated Ingredient  1876 non-null   object 
dtypes: float64(2), int64(2), object(7)
memory usage: 1.8+ MB


In [6]:
df.describe()

,Month,Illnesses,Hospitalizations,Deaths
count,19119.000000,19119.000000,15494.000000,15518.000000
mean,6.433443,19.537162,0.947528,0.021717
std,3.432970,49.452417,5.312772,0.386976
min,1.000000,2.000000,0.000000,0.000000
25%,4.000000,3.000000,0.000000,0.000000
50%,6.000000,8.000000,0.000000,0.000000
75%,9.000000,19.000000,1.000000,0.000000
max,12.000000,1939.000000,308.000000,33.000000


In [7]:
df.Deaths.value_counts()

0.0     15342
1.0       125
2.0        27
3.0         9
7.0         3
5.0         3
4.0         3
6.0         2
21.0        1
33.0        1
9.0         1
8.0         1
Name: Deaths, dtype: int64

In [8]:
missing_values_table(df)

Your selected dataframe has 11 columns and 19119 Rows.
There are 8 columns that have NULL values.


,Zero Values,null_count,% of Total Values,Total Zeroes + Null Values,% Total Zero + Null Values,Data Type
Contaminated Ingredient,0,17243,90.2,17243,90.2,object
Serotype or Genotype,0,15212,79.6,15212,79.6,object
Food Vehicle,0,8963,46.9,8963,46.9,object
Genus Species,0,6619,34.6,6619,34.6,object
Etiology Status,0,6619,34.6,6619,34.6,object
Hospitalizations,11439,3625,19.0,15064,78.8,float64
Deaths,15342,3601,18.8,18943,99.1,float64
Location of Preparation,0,1135,5.9,1135,5.9,object
Month,0,0,0.0,0,0.0,int64
State,0,0,0.0,0,0.0,object


In [9]:
df.columns.to_list()

['Month',
 'State',
 'Genus Species',
 'Serotype or Genotype',
 'Etiology Status',
 'Location of Preparation',
 'Illnesses',
 'Hospitalizations',
 'Deaths',
 'Food Vehicle',
 'Contaminated Ingredient']

In [10]:
# rename all columns
df.columns = ['month', 'state', 'genus_species', 'serotype_or_genotype', 'etiology_status', 'location_of_preparation', 'illnesses', 'hospitalizations',
             'deaths', 'food_vehicle', 'contaminated_ingredient']

In [11]:
df.head()

,month,state,genus_species,serotype_or_genotype,etiology_status,location_of_preparation,illnesses,hospitalizations,deaths,food_vehicle,contaminated_ingredient
Year,,,,,,,,,,,
2009,1,Minnesota,Norovirus,NaN,Suspected,Restaurant - Sit-down dining,2,0.0,0.0,NaN,NaN
2009,1,Minnesota,Norovirus,NaN,Confirmed,NaN,16,0.0,0.0,NaN,NaN
2009,1,Minnesota,Norovirus,NaN,Suspected,Restaurant - Sit-down dining,5,0.0,0.0,NaN,NaN
2009,1,Minnesota,Norovirus,NaN,Confirmed,"Restaurant - ""Fast-food""(drive up service or pay at counter)",3,0.0,0.0,NaN,NaN
2009,1,Minnesota,Norovirus,NaN,Confirmed,Restaurant - other or unknown type,21,0.0,0.0,cookies,NaN


In [12]:
df.genus_species.value_counts()

Norovirus Genogroup I                                          2744
Salmonella enterica                                            2303
Norovirus Genogroup II                                         1424
Norovirus unknown                                               790
Clostridium perfringens                                         732
                                                               ... 
Listeria unknown                                                  1
Norovirus Genogroup II; Norovirus                                 1
Salmonella enterica; Norovirus; Salmonella enterica               1
Campylobacter; Cryptosporidium                                    1
Bacillus cereus; Clostridium perfringens; Other - Bacterium       1
Name: genus_species, Length: 204, dtype: int64

In [13]:
df.serotype_or_genotype.value_counts()

unknown                  686
Enteritidis              682
O157:H7                  411
Typhimurium              279
Newport                  168
                        ... 
Carmel                     1
GII_4 Sydney (2012);       1
I 4,[5],12:i:-;            1
Coeln                      1
Stanley; Stanley           1
Name: serotype_or_genotype, Length: 260, dtype: int64

- will drop serotype column


In [14]:
df.contaminated_ingredient.value_counts()

fin fish                                    190
chicken                                     179
beef                                        124
egg                                         119
pork                                        105
                                           ... 
tofu                                          1
water; fin fish; sauces/ dressings            1
produce; beef                                 1
pepper, ground                                1
root (eg. potato); vine-stalk eg. tomato      1
Name: contaminated_ingredient, Length: 381, dtype: int64

- too many nulls on contaminated ingredient, this column will need to be dropped
- would have been nice to be able to use this to see if there was a commonality between illness and food type but too many blanks to work with.
- will need to focus on food vehicle and see if there is a link between illness and where food is prepared

In [15]:
df.etiology_status.value_counts()

Confirmed                                                                                                                                        7909
Suspected                                                                                                                                        4068
Suspected; Suspected                                                                                                                              310
Confirmed; Confirmed                                                                                                                              133
Confirmed; Suspected                                                                                                                               32
Confirmed; Confirmed; Confirmed                                                                                                                    17
Suspected; Confirmed                                                                                

- majority of cases were confirmed to be caused by bacteria. Will need to create a dummy's feature but first need to change if majority of cases are confirmed, then the whole column is confirmed.

In [16]:
df.head(1)

,month,state,genus_species,serotype_or_genotype,etiology_status,location_of_preparation,illnesses,hospitalizations,deaths,food_vehicle,contaminated_ingredient
Year,,,,,,,,,,,
2009,1,Minnesota,Norovirus,NaN,Suspected,Restaurant - Sit-down dining,2,0.0,0.0,NaN,NaN


In [17]:
df.food_vehicle.value_counts()

multiple foods                                           206
oysters, raw                                             165
ground beef, hamburger                                   127
salad, unspecified                                       118
chicken, unspecified                                     105
                                                        ... 
shrimp shumai                                              1
apple potato salad; spinach cheese dip                     1
mixed fruit; beets, unspecified                            1
beef, unspecified; chicken, unspecified; french fries      1
cantaloupe; honeydew melon                                 1
Name: food_vehicle, Length: 3127, dtype: int64

<h3>Notes</h3>

I need to create a unique index as year isn't going to work since it appears I have duplicate columns

In [18]:
df[df.index.duplicated()].head()

,month,state,genus_species,serotype_or_genotype,etiology_status,location_of_preparation,illnesses,hospitalizations,deaths,food_vehicle,contaminated_ingredient
Year,,,,,,,,,,,
2009,1,Minnesota,Norovirus,NaN,Confirmed,NaN,16,0.0,0.0,NaN,NaN
2009,1,Minnesota,Norovirus,NaN,Suspected,Restaurant - Sit-down dining,5,0.0,0.0,NaN,NaN
2009,1,Minnesota,Norovirus,NaN,Confirmed,"Restaurant - ""Fast-food""(drive up service or pay at counter)",3,0.0,0.0,NaN,NaN
2009,1,Minnesota,Norovirus,NaN,Confirmed,Restaurant - other or unknown type,21,0.0,0.0,cookies,NaN
2009,1,Minnesota,Norovirus,NaN,Confirmed,Restaurant - other or unknown type,7,0.0,0.0,"salad, unspecified",NaN


In [19]:
df = df.reset_index()

### NEW FEATURES

- create a column called "was_hospitalized" that creates a binary column
- create a column called "had death" that creates a binary column based on if death occured during that instance
-  create a column called "had multiple deaths" that creates a binary column based on if more than one death occured during that instance

In [20]:
df['was_hospitalized'] = np.where(df['hospitalizations']>0, 1, 0)

In [21]:
df['had_death'] = np.where(df['deaths'] > 0, 1, 0)

In [22]:
df['had_multiple_deaths'] = np.where(df['deaths']> 1, 1, 0)

-------

<h2> Use REGEX to parse the location of preparation column</h2>

Since there are multiple locations for each instance, need to split up some how. 
- splits are done using the a semi-colon
- after that is complete, then each type needs to be reduced to a couple word (i.e., 'caterer', 'restaurant', 'private home', etc)

-------

In [23]:
df.location_of_preparation.value_counts()

Restaurant - other or unknown type                                                                                                                              8125
Restaurant - Sit-down dining                                                                                                                                    2229
Private home/residence                                                                                                                                          1608
Caterer (food prepared off-site from where served)                                                                                                              1053
Other (describe in remarks)                                                                                                                                     1031
                                                                                                                                                                ... 
Restaurant

In [24]:
string = "Restaurant - other or unknown type; Private home/residence; Grocery store; Long-term care/nursing home/assisted living facility"

re.findall(r'[^;]+', string)

['Restaurant - other or unknown type',
 ' Private home/residence',
 ' Grocery store',
 ' Long-term care/nursing home/assisted living facility']

In [25]:
string = "Restaurant - Fast-food(drive up service or pay at counter); Restaurant - Sit-down dining; Private home/residence; Grocery store"
re.findall(r'[^;]+', string)


['Restaurant - Fast-food(drive up service or pay at counter)',
 ' Restaurant - Sit-down dining',
 ' Private home/residence',
 ' Grocery store']

In [26]:

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19119 entries, 0 to 19118
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Year                     19119 non-null  int64  
 1   month                    19119 non-null  int64  
 2   state                    19119 non-null  object 
 3   genus_species            12500 non-null  object 
 4   serotype_or_genotype     3907 non-null   object 
 5   etiology_status          12500 non-null  object 
 6   location_of_preparation  17984 non-null  object 
 7   illnesses                19119 non-null  int64  
 8   hospitalizations         15494 non-null  float64
 9   deaths                   15518 non-null  float64
 10  food_vehicle             10156 non-null  object 
 11  contaminated_ingredient  1876 non-null   object 
 12  was_hospitalized         19119 non-null  int64  
 13  had_death                19119 non-null  int64  
 14  had_multiple_deaths   

#### Null situation

- investigate contaminated ingredients, any way to impute?
- identify food vehicle with value counts, any way to impute?
- same goes for serotype and etiology status

In [27]:
df.food_vehicle.value_counts()

multiple foods                                           206
oysters, raw                                             165
ground beef, hamburger                                   127
salad, unspecified                                       118
chicken, unspecified                                     105
                                                        ... 
shrimp shumai                                              1
apple potato salad; spinach cheese dip                     1
mixed fruit; beets, unspecified                            1
beef, unspecified; chicken, unspecified; french fries      1
cantaloupe; honeydew melon                                 1
Name: food_vehicle, Length: 3127, dtype: int64

In [28]:
df = df.replace(to_replace='None', value=np.nan).dropna(axis=0, subset=['food_vehicle'])

df = df.dropna(axis=0, subset=['food_vehicle'])

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10156 entries, 4 to 19118
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Year                     10156 non-null  int64  
 1   month                    10156 non-null  int64  
 2   state                    10156 non-null  object 
 3   genus_species            7245 non-null   object 
 4   serotype_or_genotype     2067 non-null   object 
 5   etiology_status          7245 non-null   object 
 6   location_of_preparation  9816 non-null   object 
 7   illnesses                10156 non-null  int64  
 8   hospitalizations         7909 non-null   float64
 9   deaths                   7846 non-null   float64
 10  food_vehicle             10156 non-null  object 
 11  contaminated_ingredient  1876 non-null   object 
 12  was_hospitalized         10156 non-null  int64  
 13  had_death                10156 non-null  int64  
 14  had_multiple_deaths   

<h3>the use of REGEX below via stack</h3>

<h5>The safest way to capture the characters before the comma is: ^([^,]*)</h5>

Explanation:

- ^        Start of string
- (        Start of capture group
- [^,]*    Any number of any non-comma characters
- )        End of capture group

In [30]:
#First clean through of food_vehicle column. Creating new column called food_type. Need to clean again 
# to get rid of any items after a semi-colon
search = []    
str_re = r"^([^,]*)"
for values in df['food_vehicle']:
    search.append(re.search(str_re, values).group())



In [31]:
df['food_type'] = search


In [32]:
df.food_type.value_counts()

chicken                                696
sandwich                               551
fish                                   531
ground beef                            381
pizza                                  278
                                      ... 
salads; tomatoes; vegetables             1
cantaloupe & strawberry mix              1
cheese; deli meat                        1
cheesy chicken and rice frozen meal      1
cannelloni; chicken                      1
Name: food_type, Length: 1507, dtype: int64

### Create new column called location of preparation...home or restaurant, can make it a binary